In [1]:
import subprocess
import shlex
import os

from subprocess import PIPE
from pathlib import Path

In [2]:
def submit_monolithic_spatially_explicit_parallel(
    algorithm, config, repeats=10, walltime=1, speciation=0.000001, sample=0.00025, partitions=4, memory=16,
    reporters=['Execution()', 'Biodiversity()'], output='./STDIN'
):
    walltime = "{:02d}".format(walltime)
    reporters = ','.join(reporters)
    
    config = fr"""
    #PBS -lselect={1}:ncpus={partitions}:mem={memory}gb
    #PBS -J 0-{max(repeats - 1, 0)}
    #PBS -lwalltime={walltime}:00:00
    
    module load mpi
    module load anaconda3/personal

    mpiexec $HOME/necsim-rust/target-mpi/release/rustcoalescence simulate '(
        speciation: {speciation},
        sample: {sample},
        seed: '$(python3 -c "import random; print(random.getrandbits(64))")',

        algorithm: {algorithm}{config},

        scenario: SpatiallyExplicit(
            habitat: "'$HOME'/necsim-rust/maps/madingley/fg0size12/habitat.tif",
            dispersal: "'$HOME'/necsim-rust/maps/madingley/fg0size12/dispersal.tif",
        ),
        
        log: "'$TMPDIR'/event_log",

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-mpi/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    
    
    $HOME/necsim-rust/target-replay/release/rustcoalescence replay '(
        logs: [
            "'$(echo $TMPDIR | sed "s:\[\([0-9]\+\)\]:\[\[\]\1\[\]\]:g")'/event_log/*/*"
        ],

        reporters: [
            Plugin(
                library: "'$HOME'/necsim-rust/target-replay/release/deps/libnecsim_plugins_common.so",
                reporters: [{reporters}],
            ),
        ],
    )'
    """
    
    output = Path(output).resolve(strict=False)
    output.parent.mkdir(parents=True, exist_ok=True)
    
    cwd = os.getcwd()
    os.chdir(output.parent)
    
    # Submit the simulation batch
    result = subprocess.run(shlex.split(
        f"{os.environ['HOME']}/qsubbuf/target/release/qsubbuf -N {output.name}"
    ), check=True, input=config, stdout=PIPE, universal_newlines=True).stdout.strip()
    
    os.chdir(cwd)
    
    return result

In [4]:
for partitions in [2, 4, 8, 16, 32]:
    for (speciation, sample) in [
        (0.001, 0.00000125), (0.001, 0.0000025), (0.001, 0.00000375), (0.001, 0.000005), 
    ]:
        for delta_sync in [0.1, 0.3, 1.0, 3.2, 10.0, 31.6, 100.0, 316.2, 1000.0]:
            submit_monolithic_spatially_explicit_parallel(
                "SkippingGillespie", f"(parallelism_mode:Optimistic(delta_sync:{delta_sync}))",
                walltime=6, repeats=10, memory=62, speciation=speciation, sample=sample, partitions=partitions,
                output=f"optimistic/pbs.{partitions}.{speciation}.{sample}.{delta_sync}",
            )

In [6]:
for partitions in [2, 4, 8, 16, 32]:
    for (speciation, sample) in [
        (0.000001, 0.025), (1.0, 0.025), (0.01, 0.0025), (0.0001, 0.00025), (0.000001, 0.000025), 
    ]:
        for delta_sync in [0.01, 0.032, 0.1, 0.32, 1.0, 3.2, 10.0]:
            submit_monolithic_spatially_explicit_parallel(
                "SkippingGillespie", f"(parallelism_mode:Averaging(delta_sync:{delta_sync}))",
                walltime=6, repeats=10, memory=62, speciation=speciation, sample=sample, partitions=partitions,
                output=f"averaging/pbs.{partitions}.{speciation}.{sample}.{delta_sync}",
            )